In [1]:
import re
import shutil
import time
from pathlib import Path

from selenium.webdriver import Edge, EdgeOptions
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.remote.webelement import WebElement

In [10]:
options = EdgeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument("--disable-blink-features=AutomationControlled")

driver = Edge(options=options)
driver.set_window_rect(x=1274, y=0, width=1292, height=1398)

{'height': 1398, 'width': 1292, 'x': 1274, 'y': 0}

In [11]:
def fun_键入操作(driver: WebDriver, selector: str, value: str):
    find = driver.find_element(By.CSS_SELECTOR, selector)
    find.send_keys(Keys.CONTROL, "a")
    time.sleep(0.5)
    find.send_keys(Keys.DELETE)
    time.sleep(0.5)
    find.send_keys(value)
    time.sleep(1)


def fun_点击操作(driver: WebDriver, selector: str):
    driver.find_element(By.CSS_SELECTOR, selector).click()
    time.sleep(1)


def fun_js点击(driver: WebDriver, selector: str):
    driver.execute_script(f'document.querySelector("{selector}").click()')
    time.sleep(1)


def fun_获取数字(stem: str):
    num_list = re.findall(r"\d+", stem)
    if len(num_list) == 0:
        return 0

    return int("".join(num_list))

In [12]:
wait = WebDriverWait(driver=driver, timeout=20)
ac = ActionChains(driver=driver)

In [13]:
driver.get("https://mai.taobao.com")
wait.until(expected_conditions.title_contains("淘宝网"))

True

In [14]:
# 输入账号密码点击登录
ac.send_keys_to_element(
    driver.find_element(By.CSS_SELECTOR, "#fm-login-id"), input("用户名")
).send_keys_to_element(
    driver.find_element(By.CSS_SELECTOR, "#fm-login-password"), input("请输入密码")
).click(
    driver.find_element(By.CSS_SELECTOR, "#login-form > div.fm-btn > button")
).perform()

wait.until(expected_conditions.title_contains("千牛商家工作台"))

In [17]:
for material_path in Path(r"C:\Users\wuweihua\Desktop\UPLOAD").iterdir():
    driver.get(
        "https://item.upload.taobao.com/sell/publish.htm?catId=201160807&smartRouter=true&keyProps=%7B%7D&newRouter=1&paramCacheId=merge_router_cache_389353239_1694673279246_988&x-gpf-submit-trace-id=213e259e16946732792022149e0968"
    )

    wait.until(expected_conditions.title_contains("商品发布"))

    # 标题
    fun_键入操作(
        driver,
        "#sell-field-title > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span > span > span > input",
        material_path.stem,
    )
    # 价格
    fun_键入操作(
        driver,
        "#sell-field-price > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
        "5.8",
    )
    # 库存
    fun_键入操作(
        driver,
        "#sell-field-quantity > div.sell-component-info-wrapper-component-wrap > div > div > span > span.input-wrap > span > input",
        "888888",
    )
    # 商家编码
    fun_键入操作(
        driver,
        "#sell-field-outerId > div.sell-component-info-wrapper-component-wrap > div > div > span > span > span > span > input",
        material_path.stem,
    )

    driver.find_element(By.CSS_SELECTOR, "body").send_keys(Keys.PAGE_DOWN)

    # 上传首图
    ac.click(
        driver.find_element(
            By.CSS_SELECTOR, "#struct-mainImagesGroup > div > div > div > div > div"
        )
    ).perform()

    wait.until(
        expected_conditions.visibility_of(
            driver.find_element(
                By.CSS_SELECTOR,
                "body > div.next-overlay-wrapper.v2.opened > div > div > div > iframe",
            )
        )
    )

    # 切换到框架内
    driver.switch_to.frame(
        driver.find_element(
            By.CSS_SELECTOR,
            "body > div.next-overlay-wrapper.v2.opened > div > div > div > iframe",
        )
    )
    # bbbb 文件夹
    fun_点击操作(
        driver,
        "#sucai-tu-selector-container > div.Home_body__P3Ncj.Home_withoutHeaderBody__DxGAT > div.PicGroupList_aside__fp5JR > div > div > div > div > ul > li:nth-child(2) > div > div > div > div > span",
    )
    # 首图
    fun_点击操作(
        driver, "#sucai_tu_selector_scrollMain > div:nth-child(1) > div > label"
    )
    # 确定
    fun_点击操作(
        driver,
        "#sucai-tu-selector-container > div.Footer_contain__g740- > div:nth-child(2) > button.next-btn.next-medium.next-btn-primary.Footer_selectOk__nEl3N",
    )
    driver.switch_to.default_content()
    # 上传图片
    fun_js点击(
        driver,
        "#sell-field-descRepublicOfSell > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > div.editor--GiZhF > div > div.editPanel--nAdw5.lite-editor-panel > div.editHeader--HV9VI.sidePadding--Zi2zU > div.leftPart--RCPnT > div:nth-child(1)",
    )
    driver.switch_to.frame(
        driver.find_element(
            By.CSS_SELECTOR,
            "body > div.ln-overlay-wrapper.opened > div.ln-overlay-inner.mediaDialog--EM7Hx.ck-dialog.overlay--Bwotz > div > iframe",
        )
    )
    # 上传图片
    fun_点击操作(
        driver,
        "#container > div > div.hasEms.container > div.main > div > div.opt-body > div.list > div.search-bar > div.right-btn.btn",
    )

    # 构建图片列表
    pic_list = []
    for in_file in material_path.iterdir():
        if in_file.is_file() and in_file.suffix.lower() in [".png", ".jpg"]:
            pic_list.append(in_file)

            if len(pic_list) == 20:
                break

    pic_list.sort(key=lambda k: fun_获取数字(k.stem))

    # 上传图片
    pic_text = "\n".join([pic.as_posix() for pic in pic_list])
    driver.find_element(
        By.XPATH,
        "/html/body/div[1]/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/div[3]/input",
    ).send_keys(pic_text)
    time.sleep(len(pic_list) / 1.5)

    pic_item_list = driver.find_elements(By.CSS_SELECTOR, "#items > div")[
        : len(pic_list)
    ]

    for pic_item in pic_list:
        for item in pic_item_list:
            name = item.find_element(By.CSS_SELECTOR, ".name").text
            if name == pic_item.name:
                item.location_once_scrolled_into_view
                item.find_element(By.CSS_SELECTOR, "img").click()
                time.sleep(0.5)

    # 点击确认
    fun_点击操作(
        driver,
        "#container > div > div.hasEms.container > div.main > div > div.opt-footer > div.btn.btn-blue",
    )
    time.sleep(3)
    driver.switch_to.default_content()

    # 放入仓库
    fun_js点击(
        driver,
        "#sell-field-startTime > div.sell-component-info-wrapper-component-wrap > div.sell-component-info-wrapper-component-child-wrap > div > div > span > span:nth-child(3) > label",
    )

    # 提交宝贝
    fun_js点击(selector="#struct-buttons > button", driver=driver)

    # 删除桌面文件夹
    shutil.rmtree(material_path)

    time.sleep(10)